# Data exploration

Some preliminary questions we would like to address with the grant data we previously downloaded:

* What are the levels of funding for health innovation (as defined in the project) in the UK
* How has funding evolved over time?
* What's its geography?
* Who is supporting health innovation?
* What are its topics?
* What are example projects?
* What is Nesta doing?

We will at some point seek to compare the analysis of this funding landscape with the situation in the USA based on an analysis of the activities of RWJF.


The main goals is to identify and explore health innovation related projects. This definition has a domain aspect (the projects need to seek improvements in health outcomes) and a novelty aspect (they need to be new or different from what's done in the field). We will explore several strategies to get a handle on this. This includes:

1. Identify health projects

We will use data about project categories (which are available for some if not all projects) to train a model predicting if it is in health, and also to analyse the overlaps between projects in health and other domains.

2. Map activity inside health

Once we have a corpus of 'health' projects, we will classify them into finer categories using a third party taxonomy (eg disease areas and project types).

3. Find innovative projects

This is the least straightforward part. We are looking for novelty. This can be defined in different ways:

* Projects that mention innovation

* Projects involving innovative technologies (in this case would look for keywords based on some domain-based list of technologies or keywords.

* Projects similar to those sponsored by innovative organisations. Eg. Train a model on the Nesta data and look for similar projects

* Projects that bridge domains in unusual ways

* Projects that are unique in that they don't fall in existing clusters or form their own clusters (need to decide how to do the clustering).

* Projects with trending keywords (keywords that started appearing recently in the data)


...We will explore some of these options in the 360g data

In [1]:
%matplotlib inline
#NB I open a standard set of directories

#Paths

#Get the top path
top_path = os.path.dirname(os.getcwd())

#Create the path for external data
ext_data = os.path.join(top_path,'data/external')

#Raw path (for html downloads)

raw_data = os.path.join(top_path,'data/raw')

#And external data
proc_data = os.path.join(top_path,'data/processed')

fig_path = os.path.join(top_path,'reports/figures')

#Get date for saving files
today = datetime.datetime.today()

today_str = "_".join([str(x) for x in [today.day,today.month,today.year]])

In [120]:
#Additional imports

import ratelim

import io

## 1. Load data

In [3]:
with open(ext_data+'/22_3_2018_file_download.p','rb') as infile:
    t60_container = pickle.load(infile)

### Initial exploration

We have a collection of dataframes from different sources. They don't always share fields.
We need to determine the right set of fields to focus on for the rest of the analysis. We'll do that here.


In [4]:
#This list contains the dfs we have managed to download
t60_dfs = [x for x in t60_container if type(x)==pd.core.frame.DataFrame]

In [5]:
#What columns are shared across datasets?
t60_columns = [set([c.lower() for c in x.columns]) for x in t60_dfs]

#This gives the intersection of all the sets
u = set.intersection(*t60_columns)
u

{'currency', 'description', 'title'}

Only 3 fields are shared. We will look for the columns which appear most often in the dataset

In [6]:
#Now let's see what fields appear most frequently

def flatten_list(my_list):
    '''
    Turns a nested list into a flat list
    
    '''
    
    flat = [x for el in my_list for x in el]
    
    return(flat)

In [7]:
#This list comprehension gives us a list of column names for each df (we lower them)
t60_column_names = [[name.lower() for name in x.columns] for x in t60_dfs]

#And here are their frequencies - remember we 
column_freq = pd.Series(flatten_list(t60_column_names)).value_counts()

column_freq[:20]

currency                           86
title                              86
description                        86
award date                         82
identifier                         82
amount awarded                     82
funding org:identifier             76
funding org:name                   75
recipient org:name                 75
recipient org:identifier           75
last modified                      69
recipient org:charity number       67
planned dates:duration (months)    53
recipient org:company number       50
grant programme:title              48
planned dates:start date           47
planned dates:end date             46
beneficiary location:name          44
recipient org:postal code          41
recipient org:city                 40
dtype: int64

In [8]:
#Let's see what's the coverage with different shared column names.

def extract_shared_variables(variable_names,df_list):
    '''
    Takes a list of fields and returns a concatenated df with them.
    
    '''
    
    df_container = []
    
    for x in df_list:
        x.columns = [f.lower() for f in x.columns]
        
        x_subset = x[[var for var in x.columns if var in variable_names]]
        df_container.append(x_subset)
        
    df_concat = pd.concat(df_container,axis=0)
    
    return(df_concat)
    
        



In [9]:
#And here is the shared df!
t60_df = extract_shared_variables(column_freq[:20].index,t60_dfs)

In [10]:
#How many missing values per variable?

#This list comprehension loops over the columns and gives us the columns above a certain threshold
fields_above_thres = [val[0] for val in [(col,100*np.sum(t60_df[col].isna())/len(t60_df)) for col in t60_df.columns] if
                      val[1]<10]

fields_above_thres

['amount awarded',
 'award date',
 'currency',
 'description',
 'funding org:identifier',
 'funding org:name',
 'identifier',
 'recipient org:identifier',
 'recipient org:name',
 'title']

These are the variables present in at least 10% of projects in the data. They contain a lost of the info
we want to answer the questions above, with the exception of place :-(

In [11]:
ts_df = t60_df[fields_above_thres]

### Identify health projects

We have attempted multiple strategies to identify health projects. Some things that didn't work:

* Look for a labelled dataset with 'health labels'. There are only a few datasets in the 360d data with such labels, but their descriptions seem too short to be able to train a model predicting words associated to labels.

* Look for a vocabulary of health-related words in those projects that are labelled as 'health'. Perhaps unsurprisingly given the above, we find that most of the project descriptions are insufficiently descriptive to do this.

So we have used a crude approach instead: look for projects that mention health or similar terms identified through a word embedding analysis of the data.


In [60]:
from nltk.corpus import stopwords
import gensim
import string as stri

stop = stopwords.words('English')

#We want to keep the hyphen
punct = r'['+"".join([x for x in stri.punctuation if x!='-'])+']'



In [61]:
t60h = extract_shared_variables(fields_above_thres,t60_dfs).reset_index(drop=True)

t60h.columns = ['value','award_date','currency','description','funder_id','funder_name','identifier',
                'recipient_id','recipient_name','title']


In [74]:
#Tokenise descriptions and remove stopwords

#Levels of this list comprehension: descriptions -> words in the description if words not in stopword list after
#removing punctuation
#and np.nan if the description is a number

t60h['description_tokens'] = [
    [[w.lower().strip() for w in re.sub(punct,' ',x).split(" ") if w.lower() not in set(stop+[''])] if type(x)==str else np.nan][0] for x in t60h.description]


We start an exploration of the tokens...

In [75]:
#When calculating the word frequencies we want to ignore the elements in the list which are na
word_freq = pd.Series(flatten_list(t60h.description_tokens.dropna())).value_counts()

In [76]:
#As usual, very skewed distribution

word_freq.describe()

count     65769.000000
mean         85.793809
std        1318.853684
min           1.000000
25%           1.000000
50%           2.000000
75%           6.000000
max      140091.000000
dtype: float64

In [77]:
word_freq[:10]

project       140091
funding       105082
group          87747
community      86970
people         79764
use            69683
grant          60921
provide        55926
activities     49879
costs          44968
dtype: int64

And the words that tend to appear are not super surprising

In [78]:
#Ok - so we train a word 2 vec model in lour corpus using the default parameters

w2v = gensim.models.Word2Vec(t60h['description_tokens'].dropna())

In [79]:
#Let's get the most similar words to health
#We need a bit more pre-processing in the data

health_words = list(
    set(flatten_list([[x[0] for x in w2v.most_similar(w) if x[1]>0.5] for w in ['health','wellbeing']])))+['health',
                                                                                                          'wellbeing']

#These are the top health words
health_words


['independence',
 'well-being',
 'outcomes',
 'distress',
 'ill-health',
 'illness',
 'well',
 'diet',
 'self-esteem',
 'fitness',
 'ill',
 'overall',
 'resilience',
 'conditions',
 'health',
 'wellbeing']

In [109]:
innovation_words = list(
    set(flatten_list([[x[0] for x in w2v.most_similar(w) if x[1]>0.7] for w in ['innovation']])))+['innovation']

innovation_words

['lab', 'innovation']

In [110]:
#This list comprehension returns the number of times a health-related word appears in a description
t60h['health_terms_n'] = [np.sum(
    [x in dt for x in health_words]) if type(dt)==list else np.nan for dt in t60h['description_tokens']]

t60h['innovation_terms_n'] = [np.sum(
    [x in dt for x in innovation_words]) if type(dt)==list else np.nan for dt in t60h['description_tokens']]


In [111]:
health_innovation_projects = t60h.loc[(t60h.health_terms_n>0) & 
                           (t60h.innovation_terms_n>0),:].reset_index(drop=True)

In [113]:
health_innovation_projects.shape

(23, 13)

Not a lot!

### Let's have a look at Nesta data

In [118]:
# Focus on the Nesta data

def get_360_file(url):
    '''
    This function downloads each file in the 360 degree data. We mostly create it to decorate with the rate limiter,
    which slows down the pace at which we download files from 360.
    
    '''
    
    #Different data sources have different formatrs so we have to work with that as well.
    
    #Get the file
    request = requests.get(url)
    
    #If the status code is 200, parse etc.
    if request.status_code==200:
        
            file_type_string = get_file_type_string(request)
        
        #The parsing depends on the type of file. We get the type of file from the header or the url name
        
            #This takes ages with large files.
            if '.csv' in file_type_string:
                #We need to stream the text into the csv
                table = pd.read_csv(io.StringIO(request.text))
            
            elif '.xls' in file_type_string:
                #Excel is a bit different
                with io.BytesIO(request.content) as fh:
                    table = pd.io.excel.read_excel(fh, sheetname=0)

            elif '.json' in file_type_string:
                #There is even one download with json!
                table = pd.DataFrame.from_dict(request.json()['grants'])

            return(table)
    
    else:
        #error = requests.get(url).error
        return(request.status_code)
    
    
def get_file_type_string(request):
    '''
    This function takes the return from a webpage objec and returns a string where we look for the file type
    
    '''
    
    #Extract the url. This will often contain the file extension
    text = request.url
    
    #Also add metadata from the get, in case there was no file extension:
    if 'Content-Disposition' in request.headers:
        text = text + ' '+request.headers['Content-Disposition']
        
    return(text)


In [126]:
nesta = get_360_file('http://www.nesta.org.uk/sites/default/files/360giving_data_export_12.05.17.csv')

nesta_hl = nesta.loc[nesta['Funding Org:Department']=='Health Lab']

In [131]:
nesta_hl[['Recipient Org:Name','Description']]

,Recipient Org:Name,Description
38,Encore Futures Limited,Ageing programme grant to Encore Futures Limit...
42,Glasgow Caledonian University,Dementia Citizens programme grant to Glasgow C...
44,Book of You CIC,Dementia Citizens programme grant to Book of Y...
45,The Stroke Association,Accelerating Ideas programme grant to The Stro...
46,The British Red Cross Society,Accelerating Ideas programme grant to The Brit...
117,Turning Point,People Powered Health programme grant to Turni...
118,Lambeth Primary Care Trust,People Powered Health programme grant to Lambe...
119,Calderdale and Huddersfield NHS Foundation Trust,People Powered Health programme grant to Calde...
120,Newcastle Bridges Commissioning Consortium,People Powered Health programme grant to Newca...
121,Newcastle Bridges Commissioning Consortium,People Powered Health programme grant to Newca...
